In [ ]:
# PDF Yüklemesinin Gerçekleştirilmesi
from langchain_community.document_loaders import PyPDFLoader

loader = PyPDFLoader("why-language-models-hallucinate.pdf")
data = loader.load()

len(data)

In [ ]:
# Veriyi parçalara ayırma(Chunking İşlemi)
from langchain_text_splitters import RecursiveCharacterTextSplitter

## Chunk_Size 1000 demek her parçanın maksimum 1000 karakter uzunluğunda olacağını belirtir.
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
docs = text_splitter.split_documents(data)

print(f"Number of documents after chunking: {len(docs)}")

docs[7]

In [ ]:
# Google Generative AI Embeddings'i kullanarak Embedding Oluşturma İşlemi
from langchain_chroma import Chroma
from langchain_google_genai import GoogleGenerativeAIEmbeddings
from dotenv import load_dotenv
load_dotenv()
embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001")
vector = embeddings.embed_query("hello, world!")
vector[:5]

In [ ]:
# ChromaDB Üzerine Kayıt İşlemi
vectorstore = Chroma.from_documents(documents=docs, embedding=embeddings)
retriever = vectorstore.as_retriever(
    search_type="similarity", search_kwargs={"k": 10}
)
retieved_docs = retriever.invoke("Why do LLMs hallucinate?")
len(retieved_docs)
print(retieved_docs[5].page_content)

In [ ]:
# Google Gemini API Yapısını Kullanarak LLM Tetikleme İşlemleri
from langchain_google_genai import ChatGoogleGenerativeAI

llm = ChatGoogleGenerativeAI(
    model="gemini-1.5-pro",
    temperature=0.3,
    max_tokens=500
)
from langchain_core.prompts import ChatPromptTemplate
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.chains import create_retrieval_chain
system_prompt = (
    "You are assistant for question-answering tasks"
    "Use the following pieces of retrieved context to answer"
    "If you dont't know the answer, say that you don't know"
    "Use three sentences maximum and keep the answer concise"
    "\n\n"
    "{context}"
)

prompt = ChatPromptTemplate.from_messages(
    [
        ("system",system_prompt),
        ("human","{input}")
    ]
)

In [ ]:
# Soru-Cevap Zinciri Oluşturma(LLM+PROMPT)
question_answer_chain = create_stuff_documents_chain(llm,prompt)

In [ ]:
# RAG Zinciri Oluşturma(RAG+LLM Entegrasyonun Gerçekleşmesi)
rag_chain = create_retrieval_chain(retriever,question_answer_chain)

In [ ]:
# Kullanıcı Sorgusunu Çaıştırarak Cevap Üretme İşlemi
response = rag_chain.invoke({"input":"Why do LLMs hallucinate?"})
print(response)
print(response["answer"])

In [ ]:
# Performans Maetriklerinin Hesaplanması
from datasets import Dataset
from ragas import evaluate
from ragas.metrics import (
    faithfulness,
    answer_relevancy,
    context_precision,
    context_recall
)

In [ ]:
QUESTIONS = [
    "Why do large language models hallucinate?",
    "What causes hallucination in LLMs?",
    "How does training data affect hallucinations?",
    "What is the role of retrieval in reducing hallucination?",
    "Why are LLMs confident even when wrong?",
    "How can hallucinations be mitigated?",
    "What is hallucination in NLP?",
    "Why do generative models fabricate information?",
    "What are common sources of hallucination?",
    "How does RAG help reduce hallucinations?"
]

GROUND_TRUTHS = [
    "LLMs hallucinate because they generate text based on probabilities rather than verified facts.",
    "Hallucination occurs due to model uncertainty, lack of grounding, and training data limitations.",
    "Incomplete, noisy, or biased training data can cause hallucinations.",
    "Retrieval provides grounded context from documents, reducing hallucinations.",
    "LLMs optimize fluency, not truth, which leads to confident but incorrect answers.",
    "Hallucinations can be mitigated using retrieval, grounding, and better prompts.",
    "Hallucination is the generation of incorrect or fabricated information.",
    "Generative models predict likely text even when factual knowledge is missing.",
    "Common sources include data gaps, ambiguity, and prompt design.",
    "RAG injects external knowledge, grounding the model’s answers."
]

In [ ]:
results = []

for q, gt in zip(QUESTIONS, GROUND_TRUTHS):
    # 1) RAG zincirini 1 kez çalıştır
    resp = rag_chain.invoke({"input": q})

    # 2) Model cevabını al
    answer = resp["answer"] if isinstance(resp, dict) and "answer" in resp else str(resp)

    # 3) Aynı çalıştırmadan context’i al (mismatch olmaz)
    contexts = [d.page_content for d in resp.get("context", [])]

    if not contexts:
    contexts = [d.page_content for d in retriever.invoke(q)]

    # 4) RAGAS formatında kayıt
    results.append({
        "question": q,
        "answer": answer,
        "ground_truths": [gt],
        "contexts": contexts
    })



In [ ]:
dataset = Dataset.from_list(results)
dataset

In [ ]:
# --- RAGAS: Hakem LLM + Embedding'i açıkça Gemini/Google olarak ayarla ---

# Senin zaten kullandığın Gemini LLM nesnesi: llm
# (ör: llm = ChatGoogleGenerativeAI(...))

# Senin zaten kullandığın embedding nesnesi: embeddings
# (ör: embeddings = GoogleGenerativeAIEmbeddings(...))

try:
    # Yeni sürümlerde genelde bu importlar çalışır
    from ragas.llms import LangchainLLMWrapper
    from ragas.embeddings import LangchainEmbeddingsWrapper

    ragas_llm = LangchainLLMWrapper(llm)
    ragas_embeddings = LangchainEmbeddingsWrapper(embeddings)

except Exception as e:
    print("Wrapper importu başarısız. RAGAS sürümün farklı olabilir.")
    print("Hata:", e)
    ragas_llm = None
    ragas_embeddings = None


In [ ]:
if ragas_llm is not None and ragas_embeddings is not None:
    report = evaluate(
        dataset,
        metrics=[faithfulness, answer_relevancy, context_precision, context_recall],
        llm=ragas_llm,
        embeddings=ragas_embeddings
    )
else:
    # Wrapper yoksa eski davranışla dener (bazı sistemlerde yine çalışabilir)
    report = evaluate(
        dataset,
        metrics=[faithfulness, answer_relevancy, context_precision, context_recall]
    )


report

In [ ]:
report.to_pandas()